# VGG16 Setup for Pig Disease Classification

In [ ]:

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout


In [ ]:

# Define paths
data_dir = 'data/'

# Data preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2  # 80-20 train-validation split
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # VGG16 input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:

# Load VGG16 without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False


In [ ]:

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(train_gen.class_indices), activation='softmax')(x)  # Number of classes

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:

# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


In [ ]:

# Evaluate the model
test_loss, test_acc = model.evaluate(val_gen)
print(f"Test accuracy: {test_acc}")


In [ ]:

# Save the model for future use
model.save('vgg16_pig_disease_model.h5')


In [ ]:

# Fine-tune the model (optional)
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train again
history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5
)


In [ ]:

# Use the model for predictions on new images
from tensorflow.keras.preprocessing.image import load_img, img_to_array

img = load_img('path_to_new_image.jpg', target_size=(224, 224))
img_array = img_to_array(img) / 255.0
img_array = img_array.reshape(1, 224, 224, 3)

prediction = model.predict(img_array)
print(prediction)
